# soh estimation experimentation of Mercedes vehicles


## Setup

In [ ]:
! mkdir -p data_cache

### Imports

In [ ]:
import plotly.express as px
from scipy import stats
import numpy as np
from core.stats_utils import *
from core.pandas_utils import *
from core.config import valid_soh_points
from core.stats_utils import filter_results
from transform.fleet_info.main import fleet_info
from transform.processed_tss.main import get_processed_tss

### Data extraction

In [ ]:
tss = get_processed_tss("mercedes-benz", force_update=True)

In [ ]:
tss.columns

In [ ]:
tss['odometer'] = (
    tss.groupby('vin')['odometer']
    .apply(lambda group: group.ffill().bfill())
    .reset_index(drop=True)
   )
tss['soc'] = (
    tss.groupby('vin')['soc']
    .apply(lambda group: group.ffill().bfill())
    .reset_index(drop=True)
   )
tss['estimated_range'] = (
    tss.groupby('vin')['estimated_range']
    .apply(lambda group: group.ffill().bfill())
    .reset_index(drop=True)
   )

In [ ]:
sanity_check(tss)

In [ ]:
fleet_info.query("make == 'mercedes-benz'")["range"].value_counts(dropna=False, sort=True, ascending=False)

In [ ]:
fleet_info.query("make == 'mercedes-benz'").head()

In [ ]:
fleet_info.query("make == 'mercedes-benz' & range.isna()")[["model", "version"]].value_counts(sort=True, ascending=False).sort_index()

In [ ]:
unique_models = fleet_info.query("make == 'mercedes-benz'")["model"].unique()

for model in unique_models:
    nb_with_range = len(fleet_info.query(f"model == '{model}' & range.notna()"))
    nb_total = len(fleet_info.query(f"model == '{model}'"))
    print(f"Model: {model}")
    print(f"Number of {model} with range: {nb_with_range}")
    print(f"Number of {model}: {nb_total}")
    print(f"ratio: {nb_with_range / nb_total:.2f}")
    print()

## Time series

In [ ]:
most_common_vin = tss.groupby("vin").size().sort_values(ascending=False).idxmax()
most_common_vin
vin = "W1V44760313887086"
ts = tss.query(f"vin == '{vin}'")

In [ ]:
# px.scatter(ts, x="date", y="soc", title=f"{vin}")

In [ ]:
# px.scatter(ts, x="date", y="estimated_range", title=f"{vin}")

In [ ]:
# px.scatter(ts, x="date", y="max_range", title=f"{most_common_vin}")

In [ ]:
ts = ts.eval("estimated_range_by_soc = estimated_range/ soc").eval("estimated_range_by_soc_2 = estimated_range_discharge_loss / soc_discharge_loss").eval("estimated_range_by_soc_3 = estimated_range_discharge_loss / soc_discharge_loss") 
tss = tss.eval("estimated_range_by_soc = estimated_range / soc").eval("estimated_range_by_soc_2 = estimated_range_discharge_loss / soc_discharge_loss").eval("estimated_range_by_soc_3 = estimated_range_discharge_loss / soc_discharge_loss")

px.scatter(ts.query('soc_discharge_loss < -0.6'), x="soc_discharge_loss", y="estimated_range_by_soc_2", color="model", title=f"{vin}")
px.scatter(ts, x="soc", y="estimated_range_by_soc_3", color="model", title=f"{vin}")


#### Correlation matrice


In [ ]:
corr  = tss.corr(numeric_only=True)
selected_column = "max_range"
selected_corr = corr[[selected_column]].sort_values(by=selected_column, ascending=False)

# heat map of the correlation matrix
px.imshow(selected_corr, title=f"Correlation Matrix for {selected_column}")


#### Rolling variance


In [ ]:
# Calculation on the rolling variance 
tss["rolling_variance"] = tss.groupby("vin")["estimated_range_by_soc"].transform(lambda group: group.rolling(window=3).var())
var = tss.dropna(subset=['rolling_variance'])


In [ ]:
# Heat map 
# Sélectionner les colonnes d'intérêt
columns_of_interest = ['soc', 'estimated_range', 'estimated_range_by_soc','rolling_variance']

# Calculer la matrice de corrélation
correlation_matrix = tss[columns_of_interest].corr()
# Visualiser la matrice de corrélation avec une heatmap
fig = px.imshow(correlation_matrix,
                labels=dict(x="Variables", y="Variables", color="Correlation"),
                x=correlation_matrix.columns,
                y=correlation_matrix.index,
                title="Heatmap de la Corrélation")

# Afficher le graphique
fig.show()

## Reducing depandicies to factors


### Depandicies to soc -> Not much to do 

In [ ]:
import plotly.express as px
import numpy as np
ts_fil = tss.query("model == 'sprinter'").query('estimated_range_by_soc < 250') #.query('soc > 0.7')
# Créer le scatter plot de base
fig = px.scatter(ts_fil, 
                 x="estimated_range_by_soc",
                 y="soc", 
                 color="in_discharge_idx")

# Nettoyer les données pour la trendline
mask = (~np.isnan(ts_fil['estimated_range'])) & (~np.isnan(ts_fil['soc']))  # Enlever les NaN
x = ts_fil['estimated_range'][mask]
y = ts_fil['soc'][mask]

# Calculer la trendline avec numpy (en gérant les erreurs potentielles)
try:
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)

    # Créer des points x pour une ligne plus lisse
    x_range = np.linspace(x.min(), x.max(), 100)

    # Ajouter la trendline au graphique
    fig.add_traces(
        px.line(
            x=x_range, 
            y=p(x_range)
        ).update_traces(
            line=dict(color='black', dash='dash'),
            name='Trendline'
        ).data
    )
except np.linalg.LinAlgError:
    print("Impossible de calculer la trendline - problème avec les données")

print(z,p)

fig.show()

### Depandicies to discharging -> Much more stable in charge, we will keep only the values in charge

In [ ]:
px.scatter(tss.query("model == 'sprinter'").query('estimated_range_by_soc < 250'), 
           x="date",
           y="estimated_range_by_soc", 
           color="in_discharge")

### Correction per model


#### Vito -> Mulitoplicateur x 1/0,96

In [ ]:
# Mercedes soh
tss_filtered:DF = (
    tss.query("model == 'vito'")
    .query("in_charge_perf_mask")
    .eval("soh = estimated_range / soc / range/ 0.96 ")
    .assign(charge_size = lambda df: df.groupby(["vin", "in_charge_idx"]).transform("size"))
    .query("charge_size > 10")
)
#Applying the filter on the outliers
tss_filtered = filter_results(tss_filtered, valid_soh_points)


mask = tss_filtered['model'] == 'vito'
tss_filtered.loc[mask, 'soh'] = tss_filtered.loc[mask, 'soh'] /0.96

# Calculate average SOH and last odometer reading for each VIN
soh_per_vehicle = (
    tss_filtered
    .groupby("vin")
    .agg({
        "soh": "mean",
        "odometer": "max",
        "model": "first",
        "date": "max",
        "estimated_range": "max",
    })
    .reset_index()
)


In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Create the scatter plot
fig = px.scatter(
    soh_per_vehicle,
    x="odometer",
    y="soh",
    labels={"soh": "SoH", 'vin': 'VIN'},
    color="model",
    title="Average State-of-Health (SoH) vs Mileage",
    hover_data={"vin": True}
)

# Add a trendline for each model
for model_name, group in soh_per_vehicle.groupby('model'):
    x = group["odometer"]
    y = group["soh"]
    if len(x) > 1:  # Ensure there are enough points to fit a line
        coefficients = np.polyfit(x, y, 1)  # Linear fit (degree 1)
        trendline = np.polyval(coefficients, x)

        # Add the trendline to the plot using go.Scatter
        trendline_trace = go.Scatter(
            x=x,
            y=trendline,
            mode='lines',
            name=f'Trendline {model_name}',
            line=dict(dash='dash')  # Different dash style for each model
        )

        fig.add_trace(trendline_trace)

fig.show()

##  Final SOH

In [ ]:
tss["estimated_range"].value_counts(dropna=False, sort=True, ascending=False)


In [ ]:
# Mercedes soh
tss_filtered:DF = (
    tss
    .dropna(subset=["estimated_range"])
    .query('soc > 0.5')
    .query("in_charge_perf_mask")
    .eval("soh = estimated_range / soc / range ")
    .eval("soh_real = odometer_discharge_loss / soc_discharge_loss / range")
    .eval("soh_real_2 = estimated_range_discharge_loss / soc_discharge_loss / range")
    .assign(charge_size = lambda df: df.groupby(["vin", "in_charge_idx"]).transform("size"))
    .query("charge_size > 5")

)
#Applying filter on the number of charge 
# 1. Trouver les VINs qui ont un in_charge_idx maximum > 3
# valid_vins = (
#     tss_filtered.groupby('vin')['in_charge_idx']
#     .max()
#     .reset_index()
#     .query('in_charge_idx > 3')
#     ['vin']
# )

# 2. Filtrer le DataFrame pour ne garder que ces VINs
# tss_filtered = tss_filtered[tss_filtered['vin'].isin(valid_vins)]


# Applying correction model value 
mask = tss_filtered['model'] == 'vito'
tss_filtered.loc[mask, 'soh'] = tss_filtered.loc[mask, 'soh'] /0.97

mask = tss_filtered['model'] == 'sprinter'
tss_filtered.loc[mask, 'soh'] = tss_filtered.loc[mask, 'soh'] /0.98

#Applying the filter on the outliers
# tss_filtered = filter_results(tss_filtered, valid_soh_points)

# Calculate average SOH and last odometer reading for each VIN
soh_per_vehicle = (
    tss_filtered
    .groupby("vin")
    .agg({
        "soh": "mean",
        "soh_real": "mean",
        "soh_real_2": "mean",
        "odometer": "max",
        "model": "first",
        "date": "max",
        "estimated_range": "max",
    })
    .reset_index()
)

In [ ]:
# sanity_check(tss_filtered)

In [ ]:
soh_per_vehicle.dtypes


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Create the scatter plot
fig = px.scatter(
    soh_per_vehicle,
    x="odometer",
    y="soh_real",
    labels={"soh": "SoH", 'vin': 'VIN'},
    color="model",
    title="Average State-of-Health (SoH) vs Mileage",
    hover_data=["vin"]  # Ensure hover_data is a list of column names
)

# Add a trendline for each model
# for model_name, group in soh_per_vehicle.groupby('model'):
#     # Calculate the trendline for each group
#     x = group['odometer']
#     y = group['soh']
#     slope, intercept = np.polyfit(x, y, 1)
#     trendline = slope * x + intercept

#     # Add the trendline to the plot
#     fig.add_trace(
#         go.Scatter(
#             x=x,
#             y=trendline,
#             mode='lines',
#             name=f'Trendline {model_name}',
#             line=dict(color='black')
#         )
#     )

# Show the plot
fig.show()

In [ ]:
soh_per_vehicle['soh'] = soh_per_vehicle['soh'].round(2)
soh_per_vehicle[['vin', 'model', 'soh','odometer']].query('model == "vito" | model == "sprinter"').to_csv("soh_per_vehicle.csv", index=False)



In [ ]:
soh_per_vehicle.query("vin == 'W1V44760313886610'")

## Visualization

In [ ]:
tss["soh"] = (tss["estimated_range"] / tss["soc"]) / tss["range"] * 100
px.scatter(tss.query("vin == 'W1V44760313887086'"), 
           x="date",
           y="odometer_discharge_loss",
           color="vin")

In [ ]:
# Mercedes soh
tss_filtered_1:DF = (
    tss
    .query('soc > 0.5')
    .query('soc < 0.9')
    .query("in_charge_perf_mask")
    .eval("soh = estimated_range / soc / range ")
    .assign(charge_size = lambda df: df.groupby(["vin", "in_charge_idx"]).transform("size"))
    .query("charge_size > 5")

)

In [ ]:
px.scatter(tss_filtered_1.query("vin == 'W1V44760313887086'"), 
           x="date",
           y="soh",
           color="in_charge")

In [ ]:
corr  = ts.corr(numeric_only=True)
selected_column = "soh"
selected_corr = corr[[selected_column]].sort_values(by=selected_column, ascending=False)

# heat map of the correlation matrix
px.imshow(selected_corr, title=f"Correlation Matrix for {selected_column}")

In [ ]:
from transform.raw_tss.main import get_raw_tss
raw_tss = get_raw_tss("mercedes-benz", force_update=False)
raw_tss.columns

The resulting sohs follows the overall trend which makes a lot more sense than the previous results.  
We can assume that the informed default ranges in fleet info are wrong.

## Conclusion

Soh from estimated range seems promessing and could be used as our final resulsts to Ayvens.  
We would, however, need to improve the accuracy of the estimator.  